In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# # set working directory
# import os
# os.chdir("//content/drive/My Drive/MIOTI/PRY/Emergencias/")

# # Check current working directory.
# retval = os.getcwd()
# print ("Current working directory %s" % retval)

<img src=".\img\mioti.png">   
<br />
<center style="color:#888">Proyecto Activaciones Madrid SAMUR-PC<br/></center>

# Proyecto Activaciones Madrid SAMUR-PC: Preprocesamiento 

<img src="./img/samur690.jpg" style="width: 800px">

# Visión general del problema

* Los datos han sido obtenidos del portal del Ayuntamiento de Madrid de datos abiertos. En concreto de los datos de las activaciones de SAMUR - PC. 

https://datos.madrid.es/sites/v/index.jsp?vgnextoid=50d7d35982d6f510VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD


* Los datos se generan con una periodicdad trimestral y están repartidos en un fichero por año, desde el 2017 hasta el 2021, por lo que habrá que agragar esta información en un solo fichero.

* También como principal tarea y al ser claramante un dataset de serie temporal, hay que recuperar el día en que se sucede la activación, calulandolo apartir de las columnas que vienen como es el mes y la hora de activación.

* Otra de las tareas a realizar es la de actualizar el nombre de los 21 distritos de Madrid por su denomación oficial
* Eliminar las activaciones que no se han realizado dentro de los 21 distritos ya que no son objeto del proyecto, además de ser minoritarias.
* Se descartan las activaciones cuya hora solicitud es nan.
* Se sustituyen los nan de la columna Hospital por el literal 'Sin hospitalizacion'


* Descripción de las columnas del dataset:
     - **AÑO**: Año de la intervención
     - **MES**: Nombre del Mes
     - **HORA SOLICITUD**:  Hora del recurso activado en Emergencia. 
     - **HORA INTERVENCIÓN**: Hora de llegada al incidente.Las horas que aparecen en blanco son activaciones anuladas, bien por el propio demandante o por encontrarse un recurso más cercano.
     - **CÓDIGO**: Clasificación del incidente en función de la información del demandante.
     - **DISTRITO**: Nombre del distrito
     - **HOSPITALES**: Nombre del hospital público de traslado. Los campos en blanco se refieren a asistencias sanitarias que no finaliza con traslado hospitalario

* Mas detalles sobre el dataset:
https://datos.madrid.es/FWProjects/egob/Catalogo/Seguridad/Ficheros/Estructura_DS_Activaciones_Samur.pdf

# Importación y preparación del entorno

In [3]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = (20, 10)
rcParams.update({'font.size': 14})
from pathlib import Path

# Obtención de los datos

In [4]:
#Se realiza la carga de los ficheros que contienen las activaciones por emergencas de los 5 ultimos años.
# El año 2021 tien cargado hasta Marzo.

In [5]:
emergencias_df = pd.DataFrame()
lista_ficheros=[]
for csv_file in Path('.').glob('datos/activaciones_samur_*.csv'):
    # ToDo: rellenad esto con el código necesario para leer!
    lista_ficheros.append(pd.read_csv(csv_file,sep=';', encoding='iso8859'))
    print(csv_file)

datos\activaciones_samur_2017.csv
datos\activaciones_samur_2018.csv
datos\activaciones_samur_2019.csv
datos\activaciones_samur_2020.csv
datos\activaciones_samur_2021.csv


In [6]:
#Concatenamos todos los ficheros para hacer un dataframe único. Y resetamos el indice para que tengan uno común.
emergencias_df = pd.concat(lista_ficheros).reset_index(drop=True)
# emergencias_df['Distrito'].unique()

In [7]:
emergencias_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568717 entries, 0 to 568716
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Año                568717 non-null  int64 
 1   Mes                568717 non-null  object
 2   Hora Solicitud     562545 non-null  object
 3   Hora Intervención  513898 non-null  object
 4   Código             568550 non-null  object
 5   Distrito           563883 non-null  object
 6   Hospital           176073 non-null  object
dtypes: int64(1), object(6)
memory usage: 30.4+ MB


In [8]:
emergencias_df.sample(7)

,Año,Mes,Hora Solicitud,Hora Intervención,Código,Distrito,Hospital
474761,2020,JUNIO,10:56:57,10:59:51,Convulsión y/o epilepsia,MORATALAZ,Gregorio Marañón
427230,2019,DICIEMBRE,20:23:41,20:32:49,Patología psicosomática,VILLA DE VALLECAS,NaN
187997,2018,MAYO,14:43:18,14:54:22,Intoxicación etílica,ARGANZUELA,NaN
425961,2019,DICIEMBRE,14:45:39,14:56:44,Otros,CENTRO,Concepción (Fund. J. Díaz)
386043,2019,SEPTIEMBRE,19:22:05,19:27:41,"Casual: caída, etc",VALLECAS PTE.,NaN
565294,2021,MARZO,15:51:35,15:56:47,Patología respiratoria,SAN BLAS,NaN
215428,2018,JULIO,9:49:21,9:55:48,Intoxicación etílica,CARABANCHEL,NaN


In [9]:
emergencias_df.describe(include='all')

,Año,Mes,Hora Solicitud,Hora Intervención,Código,Distrito,Hospital
count,568717.000000,568717,562545,513898,568550,563883,176073
unique,NaN,12,84857,84427,95,26,23
top,NaN,ENERO,22:07:57,22:34:37,"Casual: caída, etc",CENTRO,Gregorio Marañón
freq,NaN,54625,33,35,96153,81470,28636
mean,2018.563234,NaN,NaN,NaN,NaN,NaN,NaN
std,1.186143,NaN,NaN,NaN,NaN,NaN,NaN
min,2017.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,2018.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,2019.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,2019.000000,NaN,NaN,NaN,NaN,NaN,NaN


# Preparar los datos para nuestro algoritmo

In [10]:
emergencias_df_aux=emergencias_df.copy()

# Definimos las relaciones entre meses y sus orden en el año
month_map = {
    'ENERO': 1,
    'FEBRERO': 2,
    'MARZO': 3,
    'ABRIL': 4,
    'MAYO': 5,
    'JUNIO': 6,
    'JULIO': 7,
    'AGOSTO': 8,
    'SEPTIEMBRE': 9,
    'OCTUBRE': 10,
    'NOVIEMBRE': 11,
    'DICIEMBRE': 12
    
}


# Añadimos la columna Mes_pp que nos servira para el preprocesamiento de la fecha.
emergencias_df_aux['Mes_pp'] = emergencias_df_aux['Mes']\
    .replace(month_map, inplace=False)

#Borramos los registros cuya Hora Solictud es NaN ya que son consideradas como activaciones erroneas.
emergencias_df_aux = emergencias_df_aux.dropna(subset=['Hora Solicitud'])
emergencias_df_aux.reset_index(inplace=True,drop=True)
emergencias_df_aux.tail(20)



,Año,Mes,Hora Solicitud,Hora Intervención,Código,Distrito,Hospital,Mes_pp
562525,2021,MARZO,22:53:51,22:59:12,"Casual: caída, etc",FUENCARRAL,NaN,3
562526,2021,MARZO,22:54:00,22:59:49,"Casual: caída, etc",BARAJAS,NaN,3
562527,2021,MARZO,23:04:12,23:11:18,Agresión sin especificar,SALAMANCA,NaN,3
562528,2021,MARZO,23:04:12,23:11:18,Agresión sin especificar,SALAMANCA,NaN,3
562529,2021,MARZO,23:05:35,23:10:33,Convulsión y/o epilepsia,SALAMANCA,NaN,3
562530,2021,MARZO,23:06:55,NaN,"Casual: caída, etc",FUENCARRAL,NaN,3
562531,2021,MARZO,23:08:51,23:21:48,Intoxicación etílica,VILLAVERDE,Doce de Octubre,3
562532,2021,MARZO,23:09:37,23:24:27,Heridas,CHAMBERI,NaN,3
562533,2021,MARZO,23:14:33,23:22:01,Agresión sin especificar,CARABANCHEL,NaN,3
562534,2021,MARZO,23:14:33,23:22:01,Agresión sin especificar,CARABANCHEL,NaN,3


In [11]:
emergencias_df_aux['Hora Solicitud_pp']=pd.to_numeric(emergencias_df_aux['Hora Solicitud'].str.split(':',expand=True)[0])
emergencias_df_aux

,Año,Mes,Hora Solicitud,Hora Intervención,Código,Distrito,Hospital,Mes_pp,Hora Solicitud_pp
0,2017,ENERO,0:23:19,0:28:59,Heridas,CENTRO,Concepción (Fund. J. Díaz),1,0
1,2017,ENERO,0:27:35,0:35:44,Agresión sin especificar,CARABANCHEL,NaN,1,0
2,2017,ENERO,0:47:26,0:55:49,Intoxicación etílica,SALAMANCA,NaN,1,0
3,2017,ENERO,0:55:13,1:02:23,Agresión sin especificar,CENTRO,Doce de Octubre,1,0
4,2017,ENERO,1:07:11,1:19:44,Accidente menos de 3 victimas,VILLA DE VALLECAS,NaN,1,1
...,...,...,...,...,...,...,...,...,...
562540,2021,MARZO,23:33:25,NaN,Intoxicación etílica,TETUAN,NaN,3,23
562541,2021,MARZO,23:34:17,23:52:36,Violencia de genero,FUENCARRAL,NaN,3,23
562542,2021,MARZO,23:47:02,23:53:46,Agresión sexual,TETUAN,La Paz,3,23
562543,2021,MARZO,23:48:18,NaN,Intoxicación etílica,CHAMBERI,NaN,3,23


In [12]:
emergencias_df_aux['Hora Solicitud_pp_shift']=pd.to_numeric(emergencias_df_aux['Hora Solicitud_pp'].shift(1))
emergencias_df_aux['Mes_pp_shift']=pd.to_numeric(emergencias_df_aux['Mes_pp'].shift(1))
emergencias_df_aux['Dia_pp']=np.nan
emergencias_df_aux

,Año,Mes,Hora Solicitud,Hora Intervención,Código,Distrito,Hospital,Mes_pp,Hora Solicitud_pp,Hora Solicitud_pp_shift,Mes_pp_shift,Dia_pp
0,2017,ENERO,0:23:19,0:28:59,Heridas,CENTRO,Concepción (Fund. J. Díaz),1,0,NaN,NaN,NaN
1,2017,ENERO,0:27:35,0:35:44,Agresión sin especificar,CARABANCHEL,NaN,1,0,0.0,1.0,NaN
2,2017,ENERO,0:47:26,0:55:49,Intoxicación etílica,SALAMANCA,NaN,1,0,0.0,1.0,NaN
3,2017,ENERO,0:55:13,1:02:23,Agresión sin especificar,CENTRO,Doce de Octubre,1,0,0.0,1.0,NaN
4,2017,ENERO,1:07:11,1:19:44,Accidente menos de 3 victimas,VILLA DE VALLECAS,NaN,1,1,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
562540,2021,MARZO,23:33:25,NaN,Intoxicación etílica,TETUAN,NaN,3,23,23.0,3.0,NaN
562541,2021,MARZO,23:34:17,23:52:36,Violencia de genero,FUENCARRAL,NaN,3,23,23.0,3.0,NaN
562542,2021,MARZO,23:47:02,23:53:46,Agresión sexual,TETUAN,La Paz,3,23,23.0,3.0,NaN
562543,2021,MARZO,23:48:18,NaN,Intoxicación etílica,CHAMBERI,NaN,3,23,23.0,3.0,NaN


* Función que infiere el día que se produce la activación partiendio de la hora y el mes.

In [13]:
def calcula_día_activacion(df):
    dia=1

    for index, row in df.iterrows(): 

        if ((row['Hora Solicitud_pp']) < (row['Hora Solicitud_pp_shift'])):
            dia += 1
            print('Cambio de dia ',dia)
        if ( ( (row['Mes_pp']) > (row['Mes_pp_shift']))|( (row['Mes_pp']==1) & (row['Mes_pp_shift']==12) ) ):
            print('Cambio de mes ', row['Mes_pp'])
            dia=1
            
            df.loc[index,'Dia_pp'] = dia
    
    return df      

In [14]:
#Se invoca a la función que calcula el día.
emergencias_df_aux=calcula_día_activacion(emergencias_df_aux)

Cambio de dia  2
Cambio de dia  3
Cambio de dia  4
Cambio de dia  5
Cambio de dia  6
Cambio de dia  7
Cambio de dia  8
Cambio de dia  9
Cambio de dia  10
Cambio de dia  11
Cambio de dia  12
Cambio de dia  13
Cambio de dia  14
Cambio de dia  15
Cambio de dia  16
Cambio de dia  17
Cambio de dia  18
Cambio de dia  19
Cambio de dia  20
Cambio de dia  21
Cambio de dia  22
Cambio de dia  23
Cambio de dia  24
Cambio de dia  25
Cambio de dia  26
Cambio de dia  27
Cambio de dia  28
Cambio de dia  29
Cambio de dia  30
Cambio de dia  31
Cambio de dia  32
Cambio de mes  2
Cambio de dia  2
Cambio de dia  3
Cambio de dia  4
Cambio de dia  5
Cambio de dia  6
Cambio de dia  7
Cambio de dia  8
Cambio de dia  9
Cambio de dia  10
Cambio de dia  11
Cambio de dia  12
Cambio de dia  13
Cambio de dia  14
Cambio de dia  15
Cambio de dia  16
Cambio de dia  17
Cambio de dia  18
Cambio de dia  19
Cambio de dia  20
Cambio de dia  21
Cambio de dia  22
Cambio de dia  23
Cambio de dia  24
Cambio de dia  25
Cambio de

Cambio de dia  32
Cambio de mes  4
Cambio de dia  2
Cambio de dia  3
Cambio de dia  4
Cambio de dia  5
Cambio de dia  6
Cambio de dia  7
Cambio de dia  8
Cambio de dia  9
Cambio de dia  10
Cambio de dia  11
Cambio de dia  12
Cambio de dia  13
Cambio de dia  14
Cambio de dia  15
Cambio de dia  16
Cambio de dia  17
Cambio de dia  18
Cambio de dia  19
Cambio de dia  20
Cambio de dia  21
Cambio de dia  22
Cambio de dia  23
Cambio de dia  24
Cambio de dia  25
Cambio de dia  26
Cambio de dia  27
Cambio de dia  28
Cambio de dia  29
Cambio de dia  30
Cambio de dia  31
Cambio de mes  5
Cambio de dia  2
Cambio de dia  3
Cambio de dia  4
Cambio de dia  5
Cambio de dia  6
Cambio de dia  7
Cambio de dia  8
Cambio de dia  9
Cambio de dia  10
Cambio de dia  11
Cambio de dia  12
Cambio de dia  13
Cambio de dia  14
Cambio de dia  15
Cambio de dia  16
Cambio de dia  17
Cambio de dia  18
Cambio de dia  19
Cambio de dia  20
Cambio de dia  21
Cambio de dia  22
Cambio de dia  23
Cambio de dia  24
Cambio de 

Cambio de dia  25
Cambio de dia  26
Cambio de dia  27
Cambio de dia  28
Cambio de dia  29
Cambio de dia  30
Cambio de dia  31
Cambio de mes  7
Cambio de dia  2
Cambio de dia  3
Cambio de dia  4
Cambio de dia  5
Cambio de dia  6
Cambio de dia  7
Cambio de dia  8
Cambio de dia  9
Cambio de dia  10
Cambio de dia  11
Cambio de dia  12
Cambio de dia  13
Cambio de dia  14
Cambio de dia  15
Cambio de dia  16
Cambio de dia  17
Cambio de dia  18
Cambio de dia  19
Cambio de dia  20
Cambio de dia  21
Cambio de dia  22
Cambio de dia  23
Cambio de dia  24
Cambio de dia  25
Cambio de dia  26
Cambio de dia  27
Cambio de dia  28
Cambio de dia  29
Cambio de dia  30
Cambio de dia  31
Cambio de dia  32
Cambio de mes  8
Cambio de dia  2
Cambio de dia  3
Cambio de dia  4
Cambio de dia  5
Cambio de dia  6
Cambio de dia  7
Cambio de dia  8
Cambio de dia  9
Cambio de dia  10
Cambio de dia  11
Cambio de dia  12
Cambio de dia  13
Cambio de dia  14
Cambio de dia  15
Cambio de dia  16
Cambio de dia  17
Cambio de 

Cambio de dia  15
Cambio de dia  16
Cambio de dia  17
Cambio de dia  18
Cambio de dia  19
Cambio de dia  20
Cambio de dia  21
Cambio de dia  22
Cambio de dia  23
Cambio de dia  24
Cambio de dia  25
Cambio de dia  26
Cambio de dia  27
Cambio de dia  28
Cambio de dia  29
Cambio de dia  30
Cambio de dia  31
Cambio de mes  10
Cambio de dia  2
Cambio de dia  3
Cambio de dia  4
Cambio de dia  5
Cambio de dia  6
Cambio de dia  7
Cambio de dia  8
Cambio de dia  9
Cambio de dia  10
Cambio de dia  11
Cambio de dia  12
Cambio de dia  13
Cambio de dia  14
Cambio de dia  15
Cambio de dia  16
Cambio de dia  17
Cambio de dia  18
Cambio de dia  19
Cambio de dia  20
Cambio de dia  21
Cambio de dia  22
Cambio de dia  23
Cambio de dia  24
Cambio de dia  25
Cambio de dia  26
Cambio de dia  27
Cambio de dia  28
Cambio de dia  29
Cambio de dia  30
Cambio de dia  31
Cambio de dia  32
Cambio de mes  11
Cambio de dia  2
Cambio de dia  3
Cambio de dia  4
Cambio de dia  5
Cambio de dia  6
Cambio de dia  7
Cambio

In [15]:
#Guardamos este primera versión intermedia csv que nos servira de base para seguir haciendo más preprocesamiento.
#emergencias_df_aux.to_csv('./datos/emergencias_pp.csv', sep=';',index=False,encoding='iso8859')

In [10]:
emergencias_df_aux=pd.read_csv('./datos/emergencias_pp.csv',sep=';', encoding='iso8859').reset_index(drop=True)
emergencias_df_aux.head()

,Año,Mes,Hora Solicitud,Hora Intervención,Código,Distrito,Hospital,Mes_pp,Hora Solicitud_pp,Hora Solicitud_pp_shift,Mes_pp_shift,Dia_pp
0,2017,ENERO,0:23:19,0:28:59,Heridas,CENTRO,Concepción (Fund. J. Díaz),1,0,NaN,NaN,1.0
1,2017,ENERO,0:27:35,0:35:44,Agresión sin especificar,CARABANCHEL,NaN,1,0,0.0,1.0,1.0
2,2017,ENERO,0:47:26,0:55:49,Intoxicación etílica,SALAMANCA,NaN,1,0,0.0,1.0,1.0
3,2017,ENERO,0:55:13,1:02:23,Agresión sin especificar,CENTRO,Doce de Octubre,1,0,0.0,1.0,1.0
4,2017,ENERO,1:07:11,1:19:44,Accidente menos de 3 victimas,VILLA DE VALLECAS,NaN,1,1,0.0,1.0,1.0


* Eliminamos algunas variables que ya no nos aportan información relevante

In [17]:
#Eliminamos algunas variables que ya no nos aportan información relevante
eliminar=['Mes','Hora Solicitud_pp','Hora Solicitud_pp_shift','Mes_pp_shift']
emergencias_df_aux=emergencias_df_aux.drop(eliminar,axis=1)
emergencias_df_aux.sample(10)

,Año,Hora Solicitud,Hora Intervención,Código,Distrito,Hospital,Mes_pp,Dia_pp
383937,2019,9:03:05,9:22:59,1 SVB,CENTRO,NaN,9,29.0
516602,2020,19:25:40,19:35:45,"Casual: caída, etc",MONCLOA,NaN,10,28.0
535548,2020,14:51:53,15:06:44,1 SVB,CENTRO,NaN,12,29.0
416736,2019,23:12:44,23:17:20,Heridas,CIUDAD LINEAL,NaN,12,14.0
183990,2018,12:04:45,12:08:29,Certificación psiquiátrico,TETUAN,NaN,5,11.0
548309,2021,19:53:43,20:02:57,Convulsión y/o epilepsia,MORATALAZ,Gregorio Marañón,2,12.0
62530,2017,19:29:10,19:40:03,Incendio,VALLECAS PTE.,NaN,6,17.0
417337,2019,9:30:52,NaN,Servicios especiales,SALAMANCA,NaN,12,16.0
125923,2017,8:11:03,8:45:18,Agresión sin especificar,TETUAN,La Paz,12,7.0
289958,2019,19:45:22,19:53:40,Patología anafiláctica,CENTRO,NaN,2,7.0


* Vamos a crear la variable de tipo fecha uniendo los campos 'Año' 'Mes_pp' 'Dia_pp' para determinar completamente la fecha de la activación. En primer lugar vamos a acomodar los formatos del campo 'Dia_pp' para poder llevar a cabo la acción

In [18]:
emergencias_df_aux['Dia_pp']=emergencias_df_aux['Dia_pp'].astype(str)
emergencias_df_aux['Dia_pp']=emergencias_df_aux['Dia_pp'].str.split('.').str[0]
emergencias_df_aux.sample(10)

,Año,Hora Solicitud,Hora Intervención,Código,Distrito,Hospital,Mes_pp,Dia_pp
11065,2017,12:07:15,12:16:38,Apertura de puerta,SALAMANCA,La Princesa,2,3
34819,2017,2:33:36,2:41:11,Otros,CENTRO,NaN,4,9
151895,2018,10:35:00,10:43:59,"Casual: caída, etc",VILLAVERDE,Doce de Octubre,2,15
255048,2018,13:30:32,NaN,Otros,CENTRO,NaN,11,11
518300,2020,20:06:55,20:15:25,Patología respiratoria,CENTRO,NaN,11,2
196195,2018,10:25:15,10:30:28,Agresión sin especificar,VILLA DE VALLECAS,NaN,6,10
487516,2020,17:02:59,NaN,Inconsciente sin filiar causa,SALAMANCA,NaN,7,23
259143,2018,20:10:27,20:21:50,"Casual: caída, etc",MORATALAZ,La Paz,11,21
209136,2018,16:23:19,16:29:14,"Casual: caída, etc",MONCLOA,NaN,7,10
521589,2020,5:13:47,5:17:26,Heridas,CENTRO,NaN,11,14


* A continuación unimos los valores de los campos y los pasamos a formato fecha. Obtenemos la fecha de activación con año mes y día.

In [19]:
emergencias_df_aux['Fecha Activacion']=emergencias_df_aux['Año'].map(str)+'-'+emergencias_df_aux['Mes_pp'].map(str)+'-'+emergencias_df_aux['Dia_pp'].map(str)
emergencias_df_aux['Fecha Activacion']=pd.to_datetime(emergencias_df_aux['Fecha Activacion'],format='%Y-%m-%d')
emergencias_df_aux.sample(10)

,Año,Hora Solicitud,Hora Intervención,Código,Distrito,Hospital,Mes_pp,Dia_pp,Fecha Activacion
36048,2017,13:08:18,13:11:38,Patología cardiovascular,CENTRO,NaN,4,12,2017-04-12
108813,2017,12:42:19,12:51:45,"Casual: caída, etc",VILLAVERDE,Doce de Octubre,10,24,2017-10-24
286767,2019,21:12:05,21:17:05,"Casual: caída, etc",CHAMARTIN,La Princesa,1,30,2019-01-30
114603,2017,10:20:43,10:28:38,"Casual: caída, etc",FUENCARRAL,Ramón y Cajal,11,8,2017-11-08
225347,2018,15:14:52,NaN,Convulsión y/o epilepsia,TETUAN,NaN,8,27,2018-08-27
26566,2017,0:20:47,0:25:02,Patología psicosomática,CENTRO,NaN,3,18,2017-03-18
282870,2019,7:29:46,7:36:23,Intoxicación etílica,USERA,NaN,1,21,2019-01-21
226566,2018,12:02:39,12:11:58,"Casual: caída, etc",USERA,NaN,9,1,2018-09-01
404881,2019,8:11:55,8:20:07,"Casual: caída, etc",CHAMARTIN,NaN,11,18,2019-11-18
425843,2020,21:18:46,21:18:51,Accidente de monopatin,CENTRO,NaN,1,5,2020-01-05


* Las activaciones que no necesitan hospitalización vienen con campo nan en la columna Hospital. Se procede a sustituir estos nan por el literal 'Sin hospitalización'

In [20]:
emergencias_df_aux['Hospital']=emergencias_df_aux['Hospital'].fillna('Sin hospitalizacion')
emergencias_df_aux 

,Año,Hora Solicitud,Hora Intervención,Código,Distrito,Hospital,Mes_pp,Dia_pp,Fecha Activacion
0,2017,0:23:19,0:28:59,Heridas,CENTRO,Concepción (Fund. J. Díaz),1,1,2017-01-01
1,2017,0:27:35,0:35:44,Agresión sin especificar,CARABANCHEL,Sin hospitalizacion,1,1,2017-01-01
2,2017,0:47:26,0:55:49,Intoxicación etílica,SALAMANCA,Sin hospitalizacion,1,1,2017-01-01
3,2017,0:55:13,1:02:23,Agresión sin especificar,CENTRO,Doce de Octubre,1,1,2017-01-01
4,2017,1:07:11,1:19:44,Accidente menos de 3 victimas,VILLA DE VALLECAS,Sin hospitalizacion,1,1,2017-01-01
...,...,...,...,...,...,...,...,...,...
562540,2021,23:33:25,NaN,Intoxicación etílica,TETUAN,Sin hospitalizacion,3,31,2021-03-31
562541,2021,23:34:17,23:52:36,Violencia de genero,FUENCARRAL,Sin hospitalizacion,3,31,2021-03-31
562542,2021,23:47:02,23:53:46,Agresión sexual,TETUAN,La Paz,3,31,2021-03-31
562543,2021,23:48:18,NaN,Intoxicación etílica,CHAMBERI,Sin hospitalizacion,3,31,2021-03-31


* Vamos a comparar los nombres de los distritos del fichero oficial con el del dataset de emergencias y unificarlos. Los distritos que no se correspondan con la ciudad de Madrid serán reemplazados por nulos y después eliminados del dataset

In [21]:
distrito_nuevo={'CENTRO':'Centro','CARABANCHEL':'Carabanchel','SALAMANCA':'Salamanca','VILLA DE VALLECAS':'Villa de Vallecas',
                'TETUAN':'Tetuán','VALLECAS PTE.':'Puente de Vallecas','LATINA':'Latina','MORATALAZ':'Moratalaz','VILLAVERDE':'Villaverde',
                'SAN BLAS':'San Blas - Canillejas','CIUDAD LINEAL':'Ciudad Lineal','MONCLOA':'Moncloa - Aravaca','CHAMBERI':'Chamberí',
                'CHAMARTIN':'Chamartín','USERA':'Usera','HORTALEZA':'Hortaleza','BARAJAS':'Barajas','ARGANZUELA':'Arganzuela',
                'RETIRO':'Retiro','FUENCARRAL':'Fuencarral - El Pardo','VICALVARO':'Vicálvaro','POZUELO':np.nan ,'FUERA TERMINO MUNICIPAL':np.nan ,
                'CARRETERAS Y CIRCUNVALACIONES':np.nan,'C.A.M.':np.nan,'LEGANES':np.nan,'PUENTE DE VALLECAS':'Puente de Vallecas',
                'MONCLOA-ARAVACA':'Moncloa - Aravaca','SAN BLAS-CANILLEJAS':'San Blas - Canillejas','FUENCARRAL-EL PARDO':'Fuencarral - El Pardo'
               }

emergencias_df_aux['Distrito'] = emergencias_df_aux['Distrito'].replace(distrito_nuevo)


In [22]:
emergencias_df_aux=emergencias_df_aux[emergencias_df_aux['Distrito'].notnull()]
emergencias_df_aux.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 556526 entries, 0 to 562544
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Año                556526 non-null  int64         
 1   Hora Solicitud     556526 non-null  object        
 2   Hora Intervención  508648 non-null  object        
 3   Código             556374 non-null  object        
 4   Distrito           556526 non-null  object        
 5   Hospital           556526 non-null  object        
 6   Mes_pp             556526 non-null  int64         
 7   Dia_pp             556526 non-null  object        
 8   Fecha Activacion   556526 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 42.5+ MB


In [23]:
#Guardamos el dataframe de emergencias con los cambios de preprocesamientos y calculo de fecha definitivo, denominación oficial de los distritos
emergencias_df_aux=emergencias_df_aux[['Fecha Activacion','Hora Solicitud','Hora Intervención','Código','Distrito','Hospital']]
emergencias_df_aux.to_csv('./datos/emergencias_pp_def.csv', sep=';',index=False,encoding='iso8859')